<a href="https://colab.research.google.com/github/parsakzr/ytu-bitirme/blob/main/Bitirme_FreeMode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets accelerate trl wandb einops
!pip install -q -U bitsandbytes

!pip -q install git+https://github.com/huggingface/peft.git

# !pip install -q trl xformers wandb datasets einops sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### Dataset
The dataset is [CodeAlpaca-20k](https://github.com/sahil280114/codealpaca)

In [ ]:
from datasets import load_dataset


dataset_id = "sahil2801/CodeAlpaca-20k"
dataset = load_dataset(dataset_id, split="train")

print(dataset)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 20022
})


### Loading the model and tokenizer

In [ ]:
import transformers
import torch


transformers.is_bitsandbytes_available()

True

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig


model_id = "Salesforce/codegen-350M-mono"

# Load the model, and quantize it in 4bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config
)

model.config.use_cache=False

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token_id = 0  # different to <eos>
tokenizer.padding_side = "left"  # Allow batched inference

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### Peft config.

In [ ]:
from peft import prepare_model_for_kbit_training
# TODO is it necessary?
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

In [ ]:
from peft import LoraConfig, get_peft_model

#Lora config
LORA_R = 4
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

peft_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
    # QLoRA paper recommended to consider ALL linear layers.
    # Llama 2 uses the layers below
    # target_modules=[
    #     "query_key_value",
    #     "dense",
    #     "dense_h_to_4h",
    #     "dense_4h_to_h",
    # ]
)

model = get_peft_model(model, peft_config)

## Dataset preperation:


In [ ]:

text = "def foo(): # This function returns the first 10 elements from Fibonacci series"
input_ids = tokenizer(text, return_tensors="pt").input_ids

generated_ids = model.generate(input_ids, max_length=128)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))


TypeError: ignored

In [ ]:
example = dataset[14]
prompt = example['instruction'] + (f"\nInput: {example['input']}" if example['input'] else '')
output = example['output']
print(f'------------ Prompt -------------\n{prompt}')
print(f'-------- Original Output --------\n{output}')
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

print('-------- Generated Output --------')
generated_ids = model.generate(input_ids, max_length=128)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

------------ Prompt -------------
Create a function to calculate the area of a given circle.
-------- Original Output --------
def area_circle(radius):
    return 3.14 * (radius**2)
-------- Generated Output --------


TypeError: ignored

In [ ]:
from trl import DataCollatorForCompletionOnlyLM

# # ref: https://huggingface.co/docs/trl/main/en/sft_trainer
# Same concept as code below but using DataCollatorForCompletionOnlyLM
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n"
        text += f"### Instruction: {example['instruction'][i]}\n\n### Input: {example['input'][i]}\n\n ### Output: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

response_template = " ### Output:"
completionCollator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [ ]:
def prompt_function(example):
  '''
  alpaca (used by e.g. gpt4-x-vicuna)

  ### Instruction: {{prompt}}
  ### Input: {{input}}
  ### Response:{{gen}}
  '''

  instruction = example['instruction']
  inputs = example['input']
  code = example['output']
  prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n"
  prompt += f"### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n{code}"

  return prompt

# # same concept but maybe used by SFTTrainer

# def formatting_prompts_func(example):
#     output_texts = []
#     for i in range(len(example['instruction'])):
#         text = f"### Question: {example['instruction'][i]}\n### Input:\n{input}\n\n ### Answer: {example['output'][i]}"
#         output_texts.append(text)
#     return output_texts

def tokenize_function(example):
  prompt = prompt_function(example)
  # example['prompt'] = prompt
  return tokenizer(prompt, truncation=True, return_tensors="pt") # , padding=False,

In [ ]:
dataset_tokenized = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names) # , num_proc=4
print(dataset_tokenized)

Parameter 'function'=<function tokenize_function at 0x7c1f9e6ab6d0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/20022 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 21
})


In [ ]:
# dataset_tokenized = dataset_tokenized.remove_columns(['instruction', 'input', 'output'])
dataset_tokenized.set_format("torch")
dataset_tokenized.column_names

['input_ids', 'attention_mask']

In [ ]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
# from datasets import *

# Dynamic Padding; Will be used as collation_fn parameter in Dataloader
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)



# Split the dataset into a training and evaluation dataset
dataset_train_test_split = dataset_tokenized.train_test_split(test_size=0.2)

# Extract the training and evaluation datasets
dataset_train = dataset_train_test_split['train']
dataset_eval = dataset_train_test_split['test']

# print(type(_train))

train_dataloader = DataLoader(
    dataset_train, shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    dataset_eval, batch_size=8, collate_fn=data_collator
)

In [ ]:
# Inspect batch health
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}


You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': torch.Size([8, 2048]), 'attention_mask': torch.Size([8, 2048])}

## Fine-tuning ⚡⚙️

In [ ]:
## TODO: uncomment if SFTTrainer didnt work
# from transformers import Trainer, TrainingArguments

# class CustomTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         # forward pass
#         outputs = model(**inputs, labels=inputs["input_ids"])
#         logits = outputs.get("logits")
#         # compute custom loss (suppose one has 3 labels with different weights)
#         loss = outputs.loss
#         return (loss, outputs) if return_outputs else loss

# The block under TrainingArguments

# trainer = CustomTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset_train,
#     eval_dataset=dataset_eval,
#     data_collator=data_collator,
#     tokenizer=tokenizer,
# )


In [ ]:
from transformers import TrainingArguments

OUTPUT_DIR = 'lora-finetuned' + model_id.split("/")[-1]
OPTIM = "paged_adamw_32bit"


# Settings for T4 (A100)
MICRO_BATCH_SIZE = 2 # 4? for A100
BATCH_SIZE = 128
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE # or just 4
EPOCHS = 1
LEARNING_RATE = 2e-4 #2e-5
CUTOFF_LEN = 256

WARMUP_RATIO = 0.03
MAX_STEPS = 100 # 300
MAX_GRAD_NORM = 0.3
LR_SCHEDULER_TYPE = "constant"
SAVE_STEPS = 50
LOGGING_STEPS = 5


training_args = TrainingArguments(
    per_device_train_batch_size=MICRO_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    num_train_epochs=EPOCHS,
    optim=OPTIM,
    warmup_ratio=WARMUP_RATIO, # warmup_steps=100,
    learning_rate=LEARNING_RATE,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    fp16=True,
    # evaluation_strategy = "steps",
    save_steps= SAVE_STEPS, # save_total_limit=3,
    output_dir=OUTPUT_DIR,
    report_to='tensorboard',
    logging_steps=LOGGING_STEPS,
    logging_dir=OUTPUT_DIR+'/logs',
    # load_best_model_at_end=True, # requires evaluation strategy and save strategy to match
)


In [ ]:
from trl import SFTTrainer

MAX_SEQ_LENGTH = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    # dataset_text_field="text", # messy error
    formatting_func=formatting_prompts_func, # description above, data preparation
    data_collator=completionCollator,
    max_seq_length=MAX_SEQ_LENGTH,
    # tokenizer=tokenizer,
    args=training_args,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/20022 [00:00<?, ? examples/s]

In [ ]:
trainer.train()
trainer.save_model()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
5,1.127200
10,1.098300
15,0.958000
20,0.919000
25,0.934200
30,0.885500
35,0.831400
40,0.818000
45,0.811100
50,0.826500


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


In [ ]:
trainer.save_model(OUTPUT_DIR)

In [ ]:
!ls

codegen-lora-finetuned-2  drive  lora-finetunedcodegen-350M-mono  results  sample_data


In [ ]:
from peft import PeftModel

model_id = "Salesforce/codegen-350M-mono"
adapters_id = OUTPUT_DIR

# Load the model, and quantize it in 4bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config
)

model.config.use_cache=False

# Load my peft config
model = PeftModel.from_pretrained(model, adapters_id)
model = model.merge_and_unload()


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:231: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
print(model)

CodeGenForCausalLM(
  (transformer): CodeGenModel(
    (wte): Embedding(51200, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-19): 20 x CodeGenBlock(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): CodeGenAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (qkv_proj): Linear4bit(in_features=1024, out_features=3072, bias=False)
          (out_proj): Linear4bit(in_features=1024, out_features=1024, bias=False)
        )
        (mlp): CodeGenMLP(
          (fc_in): Linear4bit(in_features=1024, out_features=4096, bias=True)
          (fc_out): Linear4bit(in_features=4096, out_features=1024, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=51200, bi

In [ ]:
# from transformers import AutoModel
# Inference with the new model with PEFT lora
# tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")
# !ls
# model_finetuned = AutoModel.from_pretrained("mymodel", local_files_only=True)
# Tokenizer
# tokenizer.pad_token_id = 0  # different to <eos>
# tokenizer.padding_side = "left"  # Allow batched inference

text = "Write a function to calculate the standard deviation of a list of numbers"

input_ids = tokenizer(text, return_tensors="pt").input_ids

generated_ids = model.generate(input_ids, max_length=128)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1591: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slow

Write a function to calculate the standard deviation of a list of numbers.

The function should return a float.

For example, the standard deviation of [1, 2, 3, 4, 5] is 3.141592.

Note: the population is not allowed to be empty, so the function must throw an exception if it sees an empty array.

Note: the function is designed to receive a list of numbers and returns a float.

"""

def standard_deviation(numbers):
    if len(numbers) == 0:
        raise ValueError("Empty


In [ ]:
def generate_pipeline(text):
    def generate_prompt(text):
      return f"### Question: {text} \n### Answer:"

    prompt = generate_prompt(text)
    print(f'------------ Prompt -------------\n{prompt}')

    input_ids = tokenizer(text, return_tensors="pt").input_ids
    input_ids = input_ids.to('cuda')
    generated_ids = model.generate(input_ids, max_length=128, pad_token_id=0)

    output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    print(f'-------- Generated Output --------\n{output}')

    return output

output = generate_pipeline("Create a function to calculate the area of a given circle.")

------------ Prompt -------------
### Question: Create a function to calculate the area of a given circle. 
### Answer:
-------- Generated Output --------
Create a function to calculate the area of a given circle.

The function should return the area of the circle.

The function should return 0 if the circle is not found.

The function should return the area of the circle if the circle is found.

The function should return -1 if the circle is not found.

The function should return the area of the circle if the circle is found.

The function should return the area of the circle if the circle is found.

The function should return the area of the circle if the circle is found.

The function should return the


# Model Evaluation

In [ ]:
# !git clone https://github.com/openai/human-eval.git
# !pip install -e human-eval

  # build             build everything needed to install
  # build_py          "build" pure Python modules (copy to build directory)
  # build_ext         build C/C++ extensions (compile/link to build directory)
  # build_clib        build C/C++ libraries used by Python extensions
  # build_scripts     "build" scripts (copy and fixup #! line)
  # clean             clean up temporary files from 'build' command
  # install           install everything from build directory
  # install_lib       install all Python modules (extensions and pure Python)
  # install_headers   install C/C++ header files
  # install_scripts   install scripts (Python or otherwise)
  # install_data      install data files
  # sdist             create a source distribution (tarball, zip file, etc.)
  # register          register the distribution with the Python package index
  # bdist             create a built (binary) distribution
  # bdist_dumb        create a "dumb" built distribution
  # bdist_rpm         create an RPM distribution
  # check             perform some checks on the package
  # upload            upload binary package to PyPI
%cd human-eval
!pwd
!python setup.py install
%cd ..

[Errno 2] No such file or directory: 'human-eval'
/content/human-eval
/content/human-eval
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
    

In [ ]:
from human_eval.data import write_jsonl, read_problems


#This method generates an output for the given problem using our fine-tuned model
def generate_one_completion(problems):
    print(problems)
    # feed the model with this problem
    input_ids = tokenizer(problems, truncation=True, padding=False, return_tensors="pt").input_ids
    generated_ids = model.generate(input_ids, max_length=128)
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)


#retrieve all the humanEval questions
problems = read_problems()
num_samples_per_task = 10 #200 pass@1 pass@10 ...
samples = [
    dict(task_id=task_id, completion=generate_one_completion(problems[task_id]["prompt"]))
    for task_id in problems
    for _ in range(num_samples_per_task)
]

#outputs task_id, completion pairs
write_jsonl("samples.jsonl", samples)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """



/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1591: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...


def generate_integers(a, b):
    """
    Given two positive integers a and b, return the even digits between a
    and b, in ascending order.

    For example:
    generate_integers(2, 8) => [2, 4, 6, 8]
    generate_integers(8, 2) => [2, 4, 6, 8]
    generate_integers(10, 14) => []
    """



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



def generate_integers(a, b):
    """
    Given two positive integers a and b, return the even digits between a
    and b, in ascending order.

    For example:
    generate_integers(2, 8) => [2, 4, 6, 8]
    generate_integers(8, 2) => [2, 4, 6, 8]
    generate_integers(10, 14) => []
    """



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



def generate_integers(a, b):
    """
    Given two positive integers a and b, return the even digits between a
    and b, in ascending order.

    For example:
    generate_integers(2, 8) => [2, 4, 6, 8]
    generate_integers(8, 2) => [2, 4, 6, 8]
    generate_integers(10, 14) => []
    """



In [ ]:
!evaluate_functional_correctness samples.jsonl

Traceback (most recent call last):
  File "/usr/local/bin/evaluate_functional_correctness", line 33, in <module>
    sys.exit(load_entry_point('human-eval==1.0', 'console_scripts', 'evaluate_functional_correctness')())
  File "/usr/local/bin/evaluate_functional_correctness", line 25, in importlib_load_entry_point
    return next(matches).load()
  File "/usr/lib/python3.10/importlib/metadata/__init__.py", line 171, in load
    module = import_module(match.group('module'))
  File "/usr/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib.

In [ ]:
!evaluate_functional_correctness data/example_samples.jsonl --problem_file=data/example_problem.jsonl

Traceback (most recent call last):
  File "/usr/local/bin/evaluate_functional_correctness", line 33, in <module>
    sys.exit(load_entry_point('human-eval==1.0', 'console_scripts', 'evaluate_functional_correctness')())
  File "/usr/local/bin/evaluate_functional_correctness", line 25, in importlib_load_entry_point
    return next(matches).load()
  File "/usr/lib/python3.10/importlib/metadata/__init__.py", line 171, in load
    module = import_module(match.group('module'))
  File "/usr/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib.